In [1]:
import pandas as pd
import geopandas as gp
import numpy as np 
import os
import fiona
from statistics import mean, median
import string
import sys

In [2]:
vest_or_16 = gp.read_file("./raw-from-source/VEST/or_2016/or_2016.shp")

In [3]:
vest_or_16.columns

Index(['STATE', 'COUNTY', 'PRECINCT', 'NAME', 'G16PREDCLI', 'G16PRERTRU',
       'G16PRELJOH', 'G16PREGSTE', 'G16PREOWRI', 'G16USSDWYD', 'G16USSRCAL',
       'G16USSLLIN', 'G16USSGNAV', 'G16USSWLEW', 'G16USSIREY', 'G16USSOWRI',
       'G16GOVDBRO', 'G16GOVRPIE', 'G16GOVLFOS', 'G16GOVCAUE', 'G16GOVITHO',
       'G16GOVOWRI', 'G16ATGDROS', 'G16ATGRCRO', 'G16ATGLHED', 'G16ATGOWRI',
       'G16SOSDAVA', 'G16SOSRRIC', 'G16SOSLDUR', 'G16SOSGZUN', 'G16SOSCMAR',
       'G16SOSIWEL', 'G16SOSOWRI', 'G16TREDREA', 'G16TRERGUD', 'G16TREPHEN',
       'G16TREITEL', 'G16TREOWRI', 'geometry'],
      dtype='object')

Election results from the Oregon Secretary of State's office via OpenElections (https://github.com/openelections/openelections-data-or/). Results from Polk, Tillamook, Wallowa, and Wasco counties were corrected with the respective county reports.

Precinct shapefiles for the 2016 election were obtained from the following counties: Benton, Columbia, Coos, Crook, Curry, Deschutes, Douglas, Harney, Jackson, Josephine, Klamath, Lane, Linn, Malheur, Morrow, Multnomah, Polk, Sherman, Tillamook, Umatilla, Union, Wallowa, Wasco, Washington, Yamhill.

Precinct shapefiles for the 2016 election were provided by the Oregon Secretary of State's office for the following counties: Clatsop, Hood River, Jefferson, Marion. The Marion County shapefile included numerous gaps that were assigned based on the Precinct Split Summary report from the county clerk's office. Precincts along the Salem/Kaiser border were revised with the corrected municipal boundaries from the Marion County 2019 precinct shapefile.

The 2016 precinct shapefile for Clackamas County was provided by the Metro Regional Government Data Resource Center.

Precinct boundaries for Grant County and Wheeler County are defined by school district boundaries. Precinct shapefiles were produced to match PDF maps provided by the respective counties using the 2016 Oregon Education Boundaries shapefile obtained from the Oregon Spatial Data Library.

Precinct shapefiles for the following counties were produced based on PDF maps provided by the respective counties: Baker, Gilliam, Lake, Lincoln. Boundaries defined by the PLSS grid were produced with the Oregon Public Land Survey Quarter-Quarter Reference Grid shapefile obtained from the Oregon spatial Data Library. Municipal boundaries were produced from the Oregon Department of Transportion 2016 City Limits shapefile obtained from the Oregon Spatial Data Library. Street boundaries are produced from the U.S. Census Bureau's census block shapefiles.

PDF maps obtained from Lake County are approximate precinct boundaries drawn on the PLSS grid for county precincts and on the street map for Lakeview city precincts. These boundaries were adjusted to match address range assignments in the Lake County Address Library Report which serves as the legal definition of the precincts according to the Lake County Clerk. Precinct divisions for roads that cross multiple rural precincts are generally defined by zip codes. Address ranges were further identified based on the Lake County tax lot parcel viewer.

Rural precinct boundaries in the shapefiles provided by Coos County and Union County were revised to match the updated PLSS cadastral grid in the Oregon GIS Framework obtained from the Oregon Spatial Data Library. 

Municipal precinct boundaries in the shapefiles provided by Tillamook County and Umatilla County were revised to reverse annexations that were not yet in effect for the November 2016 general election. 

The boundaries of precincts 14, 37, 45, 49 in the Deschutes County shapefile were revised to match legislative district assignments.

Precinct numbers in the Columbia County, Josephine County, and Wallowa County shapefiles were corrected to match the Oregon Secretary of State voter file.

G16PREDCLI - Hillary Clinton (Democratic Party)
G16PRERTRU - Donald Trump (Republican Party)
G16PRELJOH - Gary Johnson (Libertarian Party)
G16PREGSTE - Jill Stein (Green Party)
G16PREOWRI - Write-in Votes

G16USSDWYD - Ron Wyden (Democratic Party)
G16USSRCAL - Mark Callahan (Republican Party)
G16USSLLIN - Jim Lindsay (Libertarian Party)
G16USSGNAV - Eric Navickas (Pacfic Green Party)
G16USSWLEW - Shanti Lewallen (Working Families Party)
G16USSIREY - Steven Cody Reynolds (Independent Party)
G16USSOWRI - Write-in Votes

G16GOVDBRO - Kate Brown (Democratic Party and Working Families Party (fusion candidate))
G16GOVRPIE - Bud Pierce (Republican Party)
G16GOVLFOS - James Foster (Libertarian Party)
G16GOVCAUE - Aaron Donald Auer (Constitution Party)
G16GOVITHO - Cliff Thomason (Independent Party)
G16GOVOWRI - Write-in Votes

G16ATGDROS - Ellen Rosenblum (Democratic Party and Working Families Party (fusion candidate))
G16ATGRCRO - Daniel Zene Crowe (Republican Party)
G16ATGLHED - Lars D. H. Hedbor (Libertarian Party)
G16ATGOWRI - Write-in Votes

G16SOSDAVA - Brad Avakian (Democratic Party, Working Families, and Progressive Party (fusion candidate))
G16SOSRRIC - Dennis Richardson (Republican Party)
G16SOSLDUR - Sharon L. Durbin (Libertarian Party)
G16SOSGZUN - Alan Zundel (Pacific Green Party)
G16SOSCMAR - Michael Marsh (Constitution Party)
G16SOSIWEL - Paul Damian Wells (Independent Party)
G16SOSOWRI - Write-in Votes

G16TREDREA - Tobias Read (Democratic Party)
G16TRERGUD - Jeff Gudman (Republican Party)
G16TREPHEN - Chris Henry (Progressive Party and Pacific Green Party (fusion candidate))
G16TREITEL - Chris Telfer (Independent Party)
G16TREOWRI - Write-in Votes

In [4]:
file_list = os.listdir("./raw-from-source/Open_Elections/2016/counties")
li = []
for file in file_list:
    #print(file)
    if "general" in file:
        #print(file)
        file_ref = "./raw-from-source/Open_Elections/2016/counties/"+file
        file_prev = pd.read_csv(file_ref)
        if 'precincts' in file_prev.columns:
            file_prev['precinct'] = file_prev['precincts']
     
        file_prev = file_prev[['county', 'precinct', 'office', 'district', 'party', 'candidate','votes']]
        li.append(file_prev)
election_results = pd.concat(li, axis=0, ignore_index=True)
display(election_results)

,county,precinct,office,district,party,candidate,votes
0,Crook,1,U.S. Senate,NaN,IND,Steven C Reynolds,22
1,Crook,1,President,NaN,REP,Donald J Trump / Mike Pence,309
2,Crook,1,U.S. House,2.0,REP,Greg Walden,354
3,Crook,1,President,NaN,DEM,Hillary Clinton / Tim Kaine,122
4,Crook,1,U.S. House,2.0,DEM,James (Jim) Crary,104
...,...,...,...,...,...,...,...
72422,Sherman,3,State House,57.0,NaN,Under Votes,55
72423,Sherman,4,State House,57.0,NaN,Over Votes,0
72424,Sherman,4,State House,57.0,NaN,Under Votes,31
72425,Sherman,5,State House,57.0,NaN,Over Votes,0


In [5]:
print(election_results)

        county precinct       office  district party  \
0        Crook        1  U.S. Senate       NaN   IND   
1        Crook        1    President       NaN   REP   
2        Crook        1   U.S. House       2.0   REP   
3        Crook        1    President       NaN   DEM   
4        Crook        1   U.S. House       2.0   DEM   
...        ...      ...          ...       ...   ...   
72422  Sherman        3  State House      57.0   NaN   
72423  Sherman        4  State House      57.0   NaN   
72424  Sherman        4  State House      57.0   NaN   
72425  Sherman        5  State House      57.0   NaN   
72426  Sherman        5  State House      57.0   NaN   

                         candidate  votes  
0                Steven C Reynolds     22  
1      Donald J Trump / Mike Pence    309  
2                      Greg Walden    354  
3      Hillary Clinton / Tim Kaine    122  
4                James (Jim) Crary    104  
...                            ...    ...  
72422              

In [6]:
election_results["office"].unique()

array(['U.S. Senate', 'President', 'U.S. House', 'Governor',
       'Attorney General', 'Secretary of State', 'State Treasurer',
       'State Senate', 'State House'], dtype=object)

In [7]:
election_results = election_results[election_results["office"].isin(['U.S. Senate', 'President', 'Governor',
       'Attorney General', 'Secretary of State', 'State Treasurer'])]


In [8]:
election_results["pivot_id"] = election_results["county"].astype(str) +"-"+ election_results["precinct"].astype(str)

In [9]:
election_results

,county,precinct,office,district,party,candidate,votes,pivot_id
0,Crook,1,U.S. Senate,NaN,IND,Steven C Reynolds,22,Crook-1
1,Crook,1,President,NaN,REP,Donald J Trump / Mike Pence,309,Crook-1
3,Crook,1,President,NaN,DEM,Hillary Clinton / Tim Kaine,122,Crook-1
5,Crook,1,U.S. Senate,NaN,DEM,Ron Wyden,214,Crook-1
6,Crook,1,President,NaN,PGP,Jill Stein / Ajamu Baraka,8,Crook-1
...,...,...,...,...,...,...,...,...
72402,Sherman,3,State Treasurer,NaN,NaN,Under Votes,29,Sherman-3
72403,Sherman,4,State Treasurer,NaN,NaN,Over Votes,0,Sherman-4
72404,Sherman,4,State Treasurer,NaN,NaN,Under Votes,12,Sherman-4
72405,Sherman,5,State Treasurer,NaN,NaN,Over Votes,0,Sherman-5


In [10]:
election_results = election_results[~election_results["candidate"].isin(['Over Votes','Under Votes','Total'])]
print(election_results["candidate"].unique())

['Steven C Reynolds' 'Donald J Trump / Mike Pence'
 'Hillary Clinton / Tim Kaine' 'Ron Wyden' 'Jill Stein / Ajamu Baraka'
 'Mark Callahan' 'Gary Johnson / Bill Weld' 'Eric Navickas' 'Jim Lindsay'
 'Shanti S Lewallen' 'Write-ins' 'James Foster' 'Lars D H Hedbor'
 'Sharon L Durbin' 'Chris Henry' 'Ellen Rosenblum' 'Kate Brown'
 'Tobias Read' 'Dennis Richardson' 'Cliff Thomason' 'Paul Damian Wells'
 'Daniel Zene Crowe' 'Jeff Gudman' 'Aaron Donald Auer' 'Brad Avakian'
 'Chris Telfer' 'Alan Zundel' 'Bud Pierce' 'Michael Marsh'
 'Donald J Trump' 'Hillary Clinton' 'Jill Stein' 'Gary Johnson'
 'Donald Trump/Mike Pence' 'Gary Johnson/Bill Weld'
 'Hillary Clinton/Tim Kaine' 'Jill Stein/Ajamu Baraka' 'Donald Trump'
 'Steven Reynolds' 'Shanti Lewallen' 'Aaron Auer' 'Sharon Durbin'
 'Paul Wells' 'Lars Hedbor' 'Daniel Crowe']


In [11]:
print(election_results["pivot_id"])
election_results["cand-race"] = election_results["office"].astype(str) +"-"+ election_results["candidate"].astype(str)
print(election_results["cand-race"])

0          Crook-1
1          Crook-1
3          Crook-1
5          Crook-1
6          Crook-1
           ...    
72342    Sherman-5
72343    Sherman-5
72344    Sherman-5
72345    Sherman-5
72346    Sherman-5
Name: pivot_id, Length: 46070, dtype: object
0                U.S. Senate-Steven C Reynolds
1        President-Donald J Trump / Mike Pence
3        President-Hillary Clinton / Tim Kaine
5                        U.S. Senate-Ron Wyden
6          President-Jill Stein / Ajamu Baraka
                         ...                  
72342                U.S. Senate-Mark Callahan
72343                U.S. Senate-Eric Navickas
72344                  U.S. Senate-Jim Lindsay
72345            U.S. Senate-Shanti S Lewallen
72346                    U.S. Senate-Write-ins
Name: cand-race, Length: 46070, dtype: object


<ipython-input-11-ffd6e0560d06>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  election_results["cand-race"] = election_results["office"].astype(str) +"-"+ election_results["candidate"].astype(str)


In [12]:
cand_name_changes_dict = {'Attorney General-Daniel Crowe':"G16ATGRCRO", 
'Attorney General-Daniel Zene Crowe':"G16ATGRCRO",
'Attorney General-Ellen Rosenblum':"G16ATGDROS", 
'Attorney General-Lars D H Hedbor':"G16ATGLHED",
'Attorney General-Lars Hedbor':"G16ATGLHED",
'Attorney General-Write-ins':"G16ATGOWRI",

'Governor-Aaron Auer':"G16GOVCAUE",
'Governor-Aaron Donald Auer':"G16GOVCAUE",
'Governor-Bud Pierce':"G16GOVRPIE",
'Governor-Cliff Thomason':"G16GOVITHO",
'Governor-James Foster':"G16GOVLFOS",
'Governor-Kate Brown':"G16GOVDBRO",
'Governor-Write-ins':"G16GOVOWRI",

'President-Donald J Trump':"G16PRERTRU", 
'President-Donald J Trump / Mike Pence':"G16PRERTRU",
'President-Donald Trump':"G16PRERTRU", 
'President-Donald Trump/Mike Pence':"G16PRERTRU",
'President-Gary Johnson':"G16PRELJOH", 
'President-Gary Johnson / Bill Weld':"G16PRELJOH",
'President-Gary Johnson/Bill Weld':"G16PRELJOH",
'President-Hillary Clinton':"G16PREDCLI",
'President-Hillary Clinton / Tim Kaine':"G16PREDCLI",
'President-Hillary Clinton/Tim Kaine':"G16PREDCLI", 
'President-Jill Stein':"G16PREGSTE",
'President-Jill Stein / Ajamu Baraka':"G16PREGSTE",
'President-Jill Stein/Ajamu Baraka':"G16PREGSTE", 
'President-Write-ins':"G16PREOWRI",

'Secretary of State-Alan Zundel':"G16SOSGZUN", 
'Secretary of State-Brad Avakian':"G16SOSDAVA",
'Secretary of State-Dennis Richardson':"G16SOSRRIC",
'Secretary of State-Michael Marsh':"G16SOSCMAR", 
'Secretary of State-Paul Damian Wells':"G16SOSIWEL", 
'Secretary of State-Paul Wells':"G16SOSIWEL",
'Secretary of State-Sharon Durbin':"G16SOSLDUR",
'Secretary of State-Sharon L Durbin':"G16SOSLDUR",
'Secretary of State-Write-ins':"G16SOSOWRI",

'State Treasurer-Chris Henry':"G16TREPHEN",
'State Treasurer-Chris Telfer':"G16TREITEL",
'State Treasurer-Jeff Gudman':"G16TRERGUD",
'State Treasurer-Tobias Read':"G16TREDREA",  
'State Treasurer-Write-ins':"G16TREOWRI",

'U.S. Senate-Eric Navickas':"G16USSGNAV",
'U.S. Senate-Jim Lindsay':"G16USSLLIN",
'U.S. Senate-Mark Callahan':"G16USSRCAL", 
'U.S. Senate-Ron Wyden':"G16USSDWYD", 
'U.S. Senate-Shanti Lewallen':"G16USSWLEW",
'U.S. Senate-Shanti S Lewallen':"G16USSWLEW", 
'U.S. Senate-Steven C Reynolds':"G16USSIREY",
'U.S. Senate-Steven Reynolds':"G16USSIREY", 
'U.S. Senate-Write-ins':"G16USSOWRI"}

In [13]:
election_results["cand-race"] = election_results["cand-race"].map(cand_name_changes_dict).fillna(election_results["cand-race"])

<ipython-input-13-8cfc0d275139>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  election_results["cand-race"] = election_results["cand-race"].map(cand_name_changes_dict).fillna(election_results["cand-race"])


In [14]:
election_results_pivot = pd.pivot_table(election_results,values=["votes"],index=["pivot_id"],columns=["cand-race"],aggfunc='sum')

In [15]:
election_results_pivot.columns = election_results_pivot.columns.droplevel(0)

In [16]:
data_columns = ['G16ATGDROS', 'G16ATGLHED', 'G16ATGOWRI', 'G16ATGRCRO', 'G16GOVCAUE',
       'G16GOVDBRO', 'G16GOVITHO', 'G16GOVLFOS', 'G16GOVOWRI', 'G16GOVRPIE',
       'G16PREDCLI', 'G16PREGSTE', 'G16PRELJOH', 'G16PREOWRI', 'G16PRERTRU',
       'G16SOSCMAR', 'G16SOSDAVA', 'G16SOSGZUN', 'G16SOSIWEL', 'G16SOSLDUR',
       'G16SOSOWRI', 'G16SOSRRIC', 'G16TREDREA', 'G16TREITEL', 'G16TREOWRI',
       'G16TREPHEN', 'G16TRERGUD', 'G16USSDWYD', 'G16USSGNAV', 'G16USSIREY',
       'G16USSLLIN', 'G16USSOWRI', 'G16USSRCAL', 'G16USSWLEW']

print(len(data_columns))

34


In [17]:
election_results_pivot.columns

Index(['G16ATGDROS', 'G16ATGLHED', 'G16ATGOWRI', 'G16ATGRCRO', 'G16GOVCAUE',
       'G16GOVDBRO', 'G16GOVITHO', 'G16GOVLFOS', 'G16GOVOWRI', 'G16GOVRPIE',
       'G16PREDCLI', 'G16PREGSTE', 'G16PRELJOH', 'G16PREOWRI', 'G16PRERTRU',
       'G16SOSCMAR', 'G16SOSDAVA', 'G16SOSGZUN', 'G16SOSIWEL', 'G16SOSLDUR',
       'G16SOSOWRI', 'G16SOSRRIC', 'G16TREDREA', 'G16TREITEL', 'G16TREOWRI',
       'G16TREPHEN', 'G16TRERGUD', 'G16USSDWYD', 'G16USSGNAV', 'G16USSIREY',
       'G16USSLLIN', 'G16USSOWRI', 'G16USSRCAL', 'G16USSWLEW'],
      dtype='object', name='cand-race')

In [18]:
election_results_pivot.reset_index(drop=False,inplace=True)

In [19]:
election_results_pivot["COUNTY"] = election_results_pivot["pivot_id"].apply(lambda x: x.split("-")[0]) 
election_results_pivot["precinct"] = election_results_pivot["pivot_id"].apply(lambda x: x.split("-")[1]) 

In [21]:
#Remove the totals in Linn County
election_results_pivot = election_results_pivot[election_results_pivot["pivot_id"]!="Linn-Total"]

In [22]:
def statewide_totals_check(partner_df,source_df,column_list):
    print("***Statewide Totals Check***")
    for race in column_list:
        if (sum(partner_df[race])- source_df[race].sum() != 0):
            print(race+" has a difference of "+str(sum(partner_df[race])-source_df[race].sum())+" votes")
            print("\tVEST: "+str(sum(partner_df[race]))+" votes")
            print("\tSOURCES: "+str(source_df[race].sum())+" votes")
        else:
            print(race + " is equal", "\tVEST / RDH: " + str(sum(partner_df[race])))

In [23]:
election_results_pivot.shape
vest_or_16.shape

(1347, 39)

In [24]:
statewide_totals_check(vest_or_16,election_results_pivot,data_columns)

***Statewide Totals Check***
G16ATGDROS is equal 	VEST / RDH: 1011761
G16ATGLHED is equal 	VEST / RDH: 58609
G16ATGOWRI is equal 	VEST / RDH: 3507
G16ATGRCRO has a difference of 10 votes
	VEST: 766753 votes
	SOURCES: 766743 votes
G16GOVCAUE is equal 	VEST / RDH: 19400
G16GOVDBRO has a difference of 100 votes
	VEST: 985027 votes
	SOURCES: 984927 votes
G16GOVITHO is equal 	VEST / RDH: 47481
G16GOVLFOS is equal 	VEST / RDH: 45191
G16GOVOWRI is equal 	VEST / RDH: 3338
G16GOVRPIE has a difference of 1 votes
	VEST: 845609 votes
	SOURCES: 845608 votes
G16PREDCLI is equal 	VEST / RDH: 1002106
G16PREGSTE is equal 	VEST / RDH: 50002
G16PRELJOH is equal 	VEST / RDH: 94231
G16PREOWRI is equal 	VEST / RDH: 72594
G16PRERTRU is equal 	VEST / RDH: 782403
G16SOSCMAR is equal 	VEST / RDH: 15269
G16SOSDAVA is equal 	VEST / RDH: 814089
G16SOSGZUN is equal 	VEST / RDH: 47576
G16SOSIWEL is equal 	VEST / RDH: 64956
G16SOSLDUR is equal 	VEST / RDH: 46975
G16SOSOWRI has a difference of -2 votes
	VEST: 2646 vot

In [25]:
print(vest_or_16)

      STATE     COUNTY PRECINCT          NAME  G16PREDCLI  G16PRERTRU  \
0        41  Tillamook      033       WHEELER         204         104   
1        41  Tillamook      012     GARIBALDI         272         307   
2        41  Tillamook      024      ROCKAWAY         466         423   
3        41  Tillamook      014       KILCHIS         135         231   
4        41  Tillamook      001           BAY         350         362   
...     ...        ...      ...           ...         ...         ...   
1342     41  Tillamook      032      WESTSIDE         114          95   
1343     41    Wallowa       10    Imnaha #10          20          98   
1344     41     Marion      352           352         164         181   
1345     41  Deschutes       37  Deschutes 37         421         472   
1346     41  Deschutes       33  Deschutes 33        1140        1202   

      G16PRELJOH  G16PREGSTE  G16PREOWRI  G16USSDWYD  ...  G16SOSGZUN  \
0             18          14           8         2

In [26]:
def county_totals_check(partner_df,source_df,column_list,county_col,full_print=False):
    print("***Countywide Totals Check***")
    print("")
    diff_counties=[]
    for race in column_list:
        diff = partner_df.groupby([county_col]).sum()[race]-source_df.groupby([county_col]).sum()[race]
        for val in diff[diff != 0].index.values.tolist():
            if val not in diff_counties:
                diff_counties.append(val)
        if len(diff[diff != 0]!=0):   
            print(race + " contains differences in these counties:")
            for val in diff[diff != 0].index.values.tolist():
                county_differences = diff[diff != 0]
                print("\t"+val+" has a difference of "+str(county_differences[val])+" votes")
                print("\t\tVEST: "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
                print("\t\tSOURCES: "+str(source_df.groupby([county_col]).sum().loc[val,race])+" votes")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
        else:
            print(race + " is equal across all counties")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
    if (len(diff_counties)>0):
        print("")
        print("These are the counties with differences:")
        print(diff_counties)

In [27]:
county_totals_check(vest_or_16,election_results_pivot,data_columns,"COUNTY")

***Countywide Totals Check***

G16ATGDROS is equal across all counties
G16ATGLHED is equal across all counties
G16ATGOWRI is equal across all counties
G16ATGRCRO contains differences in these counties:
	Tillamook has a difference of 10 votes
		VEST: 5958 votes
		SOURCES: 5948 votes
G16GOVCAUE is equal across all counties
G16GOVDBRO contains differences in these counties:
	Tillamook has a difference of 100 votes
		VEST: 6152 votes
		SOURCES: 6052 votes
G16GOVITHO is equal across all counties
G16GOVLFOS is equal across all counties
G16GOVOWRI is equal across all counties
G16GOVRPIE contains differences in these counties:
	Wallowa has a difference of 1 votes
		VEST: 2837 votes
		SOURCES: 2836 votes
G16PREDCLI is equal across all counties
G16PREGSTE is equal across all counties
G16PRELJOH is equal across all counties
G16PREOWRI is equal across all counties
G16PRERTRU is equal across all counties
G16SOSCMAR is equal across all counties
G16SOSDAVA is equal across all counties
G16SOSGZUN is e

In [32]:
print(vest_or_16)
election_results_pivot.loc[:,"join_id"]=election_results_pivot.loc[:,"COUNTY"].str.upper()+"-"+election_results_pivot.loc[:,"precinct"].str.zfill(3)
vest_or_16[:,"join_id"]=vest_or_16.loc[:,"COUNTY"].str.upper()+"-"+vest_or_16.loc[:,"PRECINCT"].str.zfill(3)

      STATE     COUNTY PRECINCT          NAME  G16PREDCLI  G16PRERTRU  \
0        41  Tillamook      033       WHEELER         204         104   
1        41  Tillamook      012     GARIBALDI         272         307   
2        41  Tillamook      024      ROCKAWAY         466         423   
3        41  Tillamook      014       KILCHIS         135         231   
4        41  Tillamook      001           BAY         350         362   
...     ...        ...      ...           ...         ...         ...   
1342     41  Tillamook      032      WESTSIDE         114          95   
1343     41    Wallowa       10    Imnaha #10          20          98   
1344     41     Marion      352           352         164         181   
1345     41  Deschutes       37  Deschutes 37         421         472   
1346     41  Deschutes       33  Deschutes 33        1140        1202   

      G16PRELJOH  G16PREGSTE  G16PREOWRI  G16USSDWYD  ...  G16SOSCMAR  \
0             18          14           8         2

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


TypeError: unhashable type: 'slice'

In [33]:
final = pd.merge(vest_or_16,election_results_pivot,how="outer",on="join_id",indicator=True)
print(final["_merge"].value_counts())

final[final["_merge"]=="left_only"].to_csv("./vest_only.csv")
final[final["_merge"]=="right_only"].to_csv("./source_only.csv")

both          874
right_only    480
left_only     473
Name: _merge, dtype: int64


In [30]:
def precinct_votes_check(merged_df,column_list,vest_on_left,name_col,print_level=0):
    merged_df = merged_df.sort_values(by=[name_col],inplace=False)
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for index,row in merged_df.iterrows():
        same = True
        for i in column_list:
            left_data = i + "_x"
            right_data = i + "_y"
            if ((row[left_data] is None) or (row[right_data] is None) or (np.isnan(row[right_data])or(np.isnan(row[left_data])))):
                print("FIX NaN value at: ", row[name_col])
                return;
            diff = abs(row[left_data]-row[right_data])
            if (diff>0):
                same = False
                diff_values.append(abs(diff))
                if (diff>max_diff):
                    max_diff = diff
            if(diff>print_level):
                if (vest_on_left):
                    print(i, "{:.>72}".format(row[name_col]), "(V)","{:.>5}".format(int(row[left_data]))," (S){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))                           
                else:
                    print(i, "{:.>72}".format(row[name_col]), "(S)","{:.>5}".format(int(row[left_data]))," (V){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))
        if(same != True):
            different_rows +=1
            diff_list.append(row[name_col])
        else:
            matching_rows +=1
    print("")
    print("There are ", len(merged_df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 10")
    print("")
    print("All precincts containing differences:")
    diff_list.sort()
    print(diff_list)